# KoopKernelSequencer introduction

### Technical details

Kernels and feature maps:

We will be working in the setting of reproducing kernel Hilbert spaces (RKHS). We are using the physics-inspired bra-ket notation. Let $\phi$ be the feature function of the RKHS $\mathcal{H}$, and, for each $x\in\mathbb{R}^n$, let $\ket{\phi_x}\in\mathcal{H}$ be the associated Hilbert space element, defined by $\braket{f | \phi_x} = f(x)$, for all $f \in \mathcal{H}$. The kernel of $\mathcal{H}$ is given by $k_{x,y} = \braket{\phi_x | \phi_y}$.

Let $\lbrace x_i\rbrace_{i=1}^N \subset \mathbb{R}^n$ be the timeseries dataset, with next-time-step predicitions $\lbrace y_i\rbrace_{i=1}^N \subset \mathbb{R}^n$. The (ideal) Koopman operator $\mathcal{K}$ maps $\mathcal{K} \ket{\phi_{x_i}} = \ket{\phi_{y_i}}$ for all $i$. To the data points $x_i$ we associate orthonormal vectors $\ket{x_i}$, with $\braket{x_i | x_j} = \delta_{ij}$, in an $N$-dimensional vector space $\mathbb{R}^N_X$ with $\ket{x_i}$ as its basis. In the same way we generate a data space $\mathbb{R}_Y^N$ for the $y_i$ from basis vectors $\ket{y_i}$.

Sampling operators:

The notation follows [[1]](#1). The Koopman operator $\mathcal{K}$ on the full function space can be approximated from the given data by $\mathcal{K} = Z^\dagger W S$, where $W: \mathcal{H} \rightarrow \mathcal{H}$ is the empirical Koopman operator, and $S: \mathcal{H} \rightarrow \mathbb{R}^N$ and $Z: \mathcal{H} \rightarrow \mathbb{R}^N$ are the *sampling operators* of the inputs and the outputs. The sampling operators are Hilbert-Schmidt operators mapping between the feature space (the RKHS) $\mathcal{H}$ and the data spaces $\mathbb{R}^N_X, \mathbb{R}^N_Y$,
Using braket notaion, they can be written as $S = \sum_{i=1}^N \ket{v_{x_i}} \bra{\phi_{x_i}}$ and $S = \sum_{i=1}^N \ket{\phi_{y_i}} \bra{\phi_{y_i}}$.
They act on $f\in\mathcal{H}$ via evaluating $f$ at each point in the dataset
$$
Sf = \sum_{i=1}^N \ket{x_i} \braket{\phi_{x_i} | f} = \left( f(x_i) \right)_{i=1}^N \,,
$$
i.e. $S$ *samples* $f$ on the data points $x_i$. Note that we can naturally map the data space to the state space, $\mathbb{R}^N_X \rightarrow \mathbb{R}^n$, via $\sum_{i=1}^N \alpha_i \ket{x_i} \rightarrow \sum_{i=1}^N \alpha_i x_i$ (this mapping is not obviously neither injective nor sujective in general).

Empirical Koopman operator:

Using this terminology, we can now approximate the Koopman operator $\mathcal{K}$ by means of the sampling operators $S$ and $Z$. As shown in  [[1]](#1), the Koopman operator can written as $\mathcal{K} \approx Z^\dagger W S$ with for a $N\times N$ matrix $W$. With this, any input feature vector $\ket{\phi_{x_i}}$ is mapped to
$$
Z^\dagger W S \ket{\phi_{x_i}}
= \sum_{j,k = 1}^N \ket{\phi_{y_k}} \bra{y_k} W \ket{x_j} \braket{\phi_{x_j} | \phi_{x_i}}
= \sum_{j,k = 1}^N \ket{\phi_{y_k}} W_{kj} k_{x_j, x_i} \,.
$$
Given the next-time-step data point $y_i$ (the output), we know that the result of the mapping above sbould be given by $\ket{\phi_{y_i}}$, i.e.
$$
\sum_{j,k = 1}^N \ket{\phi_{y_k}} W_{kj} k_{x_j, x_i} = \ket{\phi_{y_i}} \,. 
$$
To relate the feature vectors on both sides to the state space $\mathrm{R}^n$, we apply again the $y$-sampling operator:
\begin{equation}
\sum_{\ell,k,j = 1}^N \ket{y_\ell} k_{y_\ell, y_k} W_{kj} k_{x_j, x_i}
= \sum_{k,j = 1}^N Z \ket{\phi_{y_k}} W_{kj} k_{x_j, x_i} 
= Z \ket{\phi_{y_i}} 
= \sum_{\ell = 1}^N \ket{y_\ell} k_{y_\ell, y_i} \,.
\end{equation}
The left- and right-hand sides can now be identified with elements of our state space $\mathrm{R}^n$ by linear extension of the mapping $\ket{y_i} \rightarrow y_i \in \mathrm{R}^n$.

Nystroem kernels and training sequencing:

The method above has an obvious shortcoming, namely the dimensionality $N\times N$ of the empirical Koopman operator $W$, where $N$ is the number of data points. Usually, the matrix $W$ is learned (or optimized) using the full dataset at once, which is not feasible for large datasets. A often used technique is using Nystroem kernels, i.e. randomly selecting a subset $\lbrace (x_i, y_i) \rbrace_{i \in I} \subset \lbrace (x_i, y_i) \rbrace_{i =1}^N$, and projecting (using the kernel $k$) the empirical Koopman operator $W$ into the spaced spanned by this subset of data points,
$$
W_{kj} = \sum_{m, n \in I} k_{y_k, y_m} \widetilde{W}_{mn} k_{x_n, x_i} \,.
$$
Using this, we only have to find the much smaller $|I| \times |I|$ matrix $\widetilde{W}$.
For our Koopman Kernel Sequencer, we use this method as well. But instead of computing $\tilde{W}$ using the techniques proposed in [[1]](#1), we implement Eq. (1) as a trainable network and train it by injecting the input-output training samples in a squential fashion. Several improvements of this basic idea are discussed below.

## References
<a id="1">[1]</a> 
Kostic, V. et al. (2022). 
Learning Dynamical Systems via Koopman Operator Regression in Reproducing Kernel Hilbert Spaces.
arXiv:2205.14027.